In [73]:
import pulp
from itertools import combinations

In [82]:
items = ["a","b","c"]
all_combinations = list(combinations(items,1))+list(combinations(items,2))+list(combinations(items,3))
large_omega = dict(zip(range(1,len(all_combinations)+1),all_combinations))
small_omega = dict(zip(range(1,len(all_combinations)+1),all_combinations))
# remove infeasible combinations at the small bin level
del(small_omega[2])
del(small_omega[5])

In [83]:
small_omega,large_omega

({1: ('a',), 3: ('c',), 4: ('a', 'b'), 6: ('b', 'c'), 7: ('a', 'b', 'c')},
 {1: ('a',),
  2: ('b',),
  3: ('c',),
  4: ('a', 'b'),
  5: ('a', 'c'),
  6: ('b', 'c'),
  7: ('a', 'b', 'c')})

In [85]:
prob = pulp.LpProblem("nested_binpacking", pulp.LpMinimize)

# x[c] = 1 <=> combination c is selected
x_large = pulp.LpVariable.dicts("x_l",large_omega.keys(),cat=pulp.LpBinary)
x_small = pulp.LpVariable.dicts("x_s",small_omega.keys(),cat=pulp.LpBinary)

# objective function to order solutions by non increasing large bins
prob += pulp.lpSum(x_large)

# set partitioning constraints
for i in items:
    prob += pulp.lpSum(x_large[c] for c in x_large if i in large_omega[c]) == 1
    prob += pulp.lpSum(x_small[c] for c in x_small if i in small_omega[c]) == 1

# consistency constraints
for s in small_omega:
    prob += x_small[s] <= pulp.lpSum(x_large[l] for l in large_omega if set(small_omega[s]).issubset(set(large_omega[l])))
for l in large_omega:
    prob += x_large[l] <= pulp.lpSum(x_small[s] for s in small_omega if set(small_omega[s]).issubset(set(large_omega[l])))

k=0
while True:
    k+=1
    status = prob.solve(solver=pulp.GUROBI_CMD(timeLimit=60*1))
    if pulp.LpStatus[status] != 'Optimal':
        break
    else:
        pos_vars = []
        zero_vars = []
        for s in x_small:
            val=pulp.value(x_small[s])
            if val>0.9:
                pos_vars.append(x_small[s])
            else:
                zero_vars.append(x_small[s])
        for s in x_large:
            val=pulp.value(x_large[s])
            if val>0.9:
                pos_vars.append(x_large[s])
            else:
                zero_vars.append(x_large[s])
        prob += pulp.lpSum((1-x) for x in pos_vars)+pulp.lpSum(x for x in zero_vars) >= 1
        print("====================================================================")
        print(k)
        for t in large_omega:
             if pulp.value(x_large[t])>0.9:
                 print("large bin",large_omega[t])
                 for c in small_omega:
                    if pulp.value(x_small[c])>0.9 and set(small_omega[c]).issubset(set(large_omega[t])):
                        print("   small bin",small_omega[c])

1
large bin ('a', 'b', 'c')
   small bin ('c',)
   small bin ('a', 'b')
2
large bin ('a', 'b', 'c')
   small bin ('a',)
   small bin ('b', 'c')
3
large bin ('a', 'b', 'c')
   small bin ('a', 'b', 'c')
4
large bin ('c',)
   small bin ('c',)
large bin ('a', 'b')
   small bin ('a', 'b')
5
large bin ('a',)
   small bin ('a',)
large bin ('b', 'c')
   small bin ('b', 'c')
